## Analyze the sentiment of Twitter users

1.   Apply the NLP techniques to build a model that can categorize whether a sentiment is positive or negative
2.   Take appropriate to reduce negative experience.



Begin with downloading the basic libraries

In [ ]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
import warnings
warnings.filterwarnings("ignore")
import nltk
import sqlite3

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn import metrics 
from sklearn.metrics import roc_curve,auc

In [ ]:
#Load the dataset

In [ ]:
df = pd.read_csv("/content/LabeledText.csv", encoding='latin-1')


In [ ]:
df.sample(5)

,File Name,Caption,LABEL
2298,3283.txt,Who the fuck actually uses this app. Why does ...,negative
1032,1928.txt,"RT @StevieNicks: ""They represent my life behin...",negative
4267,5073.txt,RT @FFArsenal: Arsenal have first bid rejected...,negative
1736,2755.txt,??? ??????? ????? ??? ???? ??? ??? ?? ¡­,neutral
1145,2029.txt,Cliff dwelling above Peacocks Gallop (recently...,neutral


In [ ]:
df['LABEL'].value_counts()

neutral     1771
positive    1646
negative    1452
Name: LABEL, dtype: int64

In [ ]:
df = df[df['LABEL'] != 'neutral']

In [ ]:
df["LABEL"] =df["LABEL"].map({'negative':0,'positive':1})

In [ ]:
df.head()

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,0
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,0
2,100.txt,This is my Valentine's from 1 of my nephews. I...,1
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,1
5,1002.txt,Chaotic Love - giclee print ?65 at #art #love ...,1


In [ ]:
df.head()

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,0
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,0
2,100.txt,This is my Valentine's from 1 of my nephews. I...,1
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,1
5,1002.txt,Chaotic Love - giclee print ?65 at #art #love ...,1


In [ ]:
df = df.drop(["File Name"],axis=1)

In [ ]:
def textpreprocessing(text):
  text = text.lower()
  text = re.sub(r"\'t","don't",text)
  text = re.sub(r"I've", "I have", text)
  text = re.sub(r"I'm", "I am", text)
  text = re.sub(r"didn't", "did not", text)
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"(@.*?)[\s]"," ", text)
  text = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', text)
  text = re.sub(r'[^\w\s]'," ",text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text



In [ ]:
df["Caption"] = df["Caption"].apply(textpreprocessing)

In [ ]:
df.sample(5)

,Caption,LABEL
4179,rt the enemy s tom clarke on their new album t...,1
3644,rt risking the ecstatic 60 x72 oil on canvas p...,1
2472,rt mexico s grajales overjoyed with triathlon ...,1
1248,wittzparty renae has the power to murder witho...,0
2279,rt this is very cute blake lively photo kissin...,1


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

List the stop words in-order to reduce features as we will be using Tf_IDF

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:

stopwords =set(['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves'])

Import Lemmatizer and Beautiful Soup for Text pre-processing

In [ ]:
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()

Sort everything using tqdm and store as list

In [ ]:
from tqdm import tqdm
preprocessed_reviews = []

for sentence in tqdm(df["Caption"].values):
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = lemmatizer.lemmatize(sentence)
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower not in stopwords)
    preprocessed_reviews.append(sentence.strip())

100%|██████████| 3098/3098 [00:01<00:00, 1817.41it/s]


In [ ]:
preprocessed_reviews[500]

'february winter rainy stormy windy tuesday night love happy positive passionate calm joy coffee'

Split tha data into X and Y

In [ ]:
x =df["Caption"]
y = df["LABEL"]

Train X and Y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=101)

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  

Model Building:

1. Random Forest
2. Naive Bayes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipe = Pipeline([("TfIdf", TfidfVectorizer()),
 ('classifier', RandomForestClassifier())])

In [ ]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('TfIdf', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [ ]:
# predict for evaluation
y_pred_train = pipe.predict(x_train)
y_pred_test = pipe.predict(x_test)

In [ ]:
# Evaluate the model - use performance matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
print(confusion_matrix(y_test, y_pred_test))

[[298  44]
 [ 94 339]]


In [ ]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.76      0.87      0.81       342
           1       0.89      0.78      0.83       433

    accuracy                           0.82       775
   macro avg       0.82      0.83      0.82       775
weighted avg       0.83      0.82      0.82       775



In [ ]:
print(np.round(accuracy_score(y_test, y_pred_test),decimals =2))

0.82


In [ ]:
#NaiveBayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
pipe = Pipeline([("TfIdf", TfidfVectorizer()),
 ('classifier', MultinomialNB(alpha=1.8))])

In [ ]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('TfIdf', TfidfVectorizer()),
                ('classifier', MultinomialNB(alpha=1.8))])

In [ ]:
# predict for evaluation
y_pred_train = pipe.predict(x_train)
y_pred_test = pipe.predict(x_test)

In [ ]:
print(confusion_matrix(y_test, y_pred_test))

[[266  76]
 [ 29 404]]


In [ ]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.90      0.78      0.84       342
           1       0.84      0.93      0.88       433

    accuracy                           0.86       775
   macro avg       0.87      0.86      0.86       775
weighted avg       0.87      0.86      0.86       775



In [ ]:
print(np.round(accuracy_score(y_test, y_pred_test),decimals =2))

0.86


Word2Vec

Use Word 2 vec. Here I will be using word2vec for similar text identification for analysis instead of feature reduction

In [ ]:
import gensim
from gensim.models import Word2Vec,KeyedVectors

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=996f77760362c6b140b9d06b0213270e39e2ba9fcaec2eea310010f53eaa5d99
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
list_of_sentence = []
for sentence in preprocessed_reviews:
    list_of_sentence.append(sentence.split())

In [ ]:
model = gensim.models.Word2Vec(window=10, min_count= 2, workers=4)
model.build_vocab(list_of_sentence, progress_per = 1000)
model.train(list_of_sentence, total_examples = model.corpus_count, epochs=model.epochs)

(143251, 219690)

In [ ]:
is_your_ram_gt_16g = False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
  model = Word2Vec(list_of_sentence, window=10, min_count=2, workers=4)
  print(model.wv.most_similar('yes'))
  print("="*20)
  print(model.wv.most_similar('excellent'))

elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(model.wv.most_similar('yes'))
        print("="*20)
        print(model.wv.most_similar('excellent'))
    else:
        print("You don't have google's word2vec file, keep want_to_train_w2v = True, to train your own model")


[('make', 0.9937700629234314), ('let', 0.9937164187431335), ('be', 0.9937081933021545), ('real', 0.9936731457710266), ('via', 0.993657112121582), ('ó', 0.9936562776565552), ('high', 0.9936429858207703), ('walking', 0.9936186671257019), ('is', 0.9936128854751587), ('exposed', 0.9936001300811768)]
[('poor', 0.9893402457237244), ('furious', 0.9892243146896362), ('ìù¾', 0.9891378879547119), ('7', 0.9890175461769104), ('top', 0.9889020919799805), ('pink', 0.9888662099838257), ('light', 0.9888626337051392), ('à', 0.9888274073600769), ('zippy', 0.9888195991516113), ('way', 0.9888079762458801)]


Thank You!!!!!!